In [1]:
import stackprinter  # type: ignore
import jupyter_black  # type: ignore
from dotenv import load_dotenv  # type: ignore
import time
import string

from baml_agents import init_logging, with_model
from baml_client.async_client import b

init_logging()
stackprinter.set_excepthook()
load_dotenv()
jupyter_black.load()

b = with_model(b, "gpt-4.1")

# Using BAML in Jupyter notebokos

## 1. Streaming outputs

BAML offers a great developer experience, and I used some utilities to recreate some of the magic in Jupyter notebooks.

In [ ]:
from baml_agents import JupyterOutputBox

with JupyterOutputBox() as box:
    output = ""
    for letter in string.ascii_lowercase * 3:
        output += letter
        box.update(output)
        time.sleep(1e-2)

# RUnning the above code will show a live-updating output box in Jupyter Notebook.
# It prints the letters of the alphabet in a box, one by one, with a small delay between each letter.

Running it simply, we have to wait for the output to finish before we can see it. Not great:

In [ ]:
await b.Four_WriteShortStory()

Now we can see the result as it's being generated:

In [6]:
from baml_agents import JupyterBamlCollector


with JupyterOutputBox(clear_after_finish=False) as s:
    c = JupyterBamlCollector(b, stream_callback=s.display(formatter="json"))

    await c.b.Four_WriteShortStory()

{
 "genre": "science fiction",
 "characters": [
 {
 "name": "Lina",
 "age": 28,
 "occupation": "starship engineer"
 },
 {
 "name": "Dax",
 "age": 35,
 "occupation": "pilot"
 },
 {
 "name": "Hira",
 "age": 42,
 "occupation": "botanist"
 }
 ],
 "story_summary": "Lost in a cosmic storm, Lina repairs the ship while Dax searches for nearby planets and Hira nurtures the last oxygen plant, hope flickering amid starlit uncertainty."
}